Домашняя работа
по теме «Работа с РСУБД. PostgreSQL / MySQL + Python. pySpark
фреймворк»

Задача 1 Создание базы данных из файлов и работа с
реляционными СУБД

In [1]:
# Параметры подключения к базе данных
db_config = {
    'host': 'localhost', # хост
    'user': 'root', # имя пользователя
    'password': 'parol' # пароль
} 
db_name = 'bike_store' #  Название базы данных

In [2]:
# Распаковка архива
from tools import unpack_zip # Импортируем функцию unpack_zip из файла tools
unpack_zip('archive.zip', 'data')

Содержимое archive.zip успешно распаковано в папку data.


In [3]:
# Создаем базу данных
from tools import manager_db # Импортируем функцию manager_db из файла tools
manager_db(db_name, 'del', **db_config) # Удаляем базу учитывая количество экспериментов
manager_db(db_name, 'add', **db_config) # Создаем новую базу данных

Соединение установлено
База данных "bike_store" успешно удалена.
Соединение закрыто
Соединение установлено
База данных "bike_store" успешно создана.
Соединение закрыто


In [4]:
# Импортируем данные, устанавливаем первичные ключи и связи между таблицами
from tools import import_data # Импортируем функцию import_data из файла tools
import_data(db_name, 'data', **db_config)
# PS не успел узнать как установить PK и FK одновременно на одно поле из python

Первичный ключ для таблицы brands успешно создан
Первичный ключ для таблицы categories успешно создан
Первичный ключ для таблицы customers успешно создан
Первичный ключ для таблицы order_items успешно создан
Первичный ключ для таблицы orders успешно создан
Первичный ключ для таблицы products успешно создан
Первичный ключ для таблицы staffs успешно создан
Первичный ключ для таблицы stocks успешно создан
Первичный ключ для таблицы stores успешно создан
Связь установлена между brands и products по полю brand_id
Связь установлена между categories и products по полю category_id
Связь установлена между customers и orders по полю customer_id
Связь установлена между orders и order_items по полю order_id
Связь установлена между products и order_items по полю product_id
Связь установлена между products и stocks по полю product_id
Связь установлена между staffs и orders по полю staff_id
Связь установлена между stores и orders по полю store_id
Связь установлена между stores и staffs по полю store_

In [5]:
 # Подключение к базе данных MySQL
import pymysql
from sqlalchemy import create_engine, text
 
connection_string = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_name}"
engine = create_engine(connection_string)

In [6]:
# Запрос №1 Список названий продуктов и соответствующие им торговые марки (brand)
with engine.connect() as connection:
    query = text("""
        SELECT
            p.product_name, 
            (SELECT b.brand_name from brands AS b WHERE b.brand_id = p.brand_id) AS brand_name 
        FROM 
            products AS p 
        ORDER BY  
            brand_name
    """)  # Запрос SQL
    result = connection.execute(query)  # Выполняем запрос
    
    # Выводим результаты на экран
    for row in result:
        print(row)

("Electra Amsterdam Fashion 7i Ladies' - 2017", 'Electra')
("Electra Townie Original 7D EQ Ladies' - 2017/2018", 'Electra')
('Electra Townie Original 7D EQ - 2018', 'Electra')
("Electra Townie Original 3i EQ Ladies' - 2018", 'Electra')
('Electra Townie Original 3i EQ - 2017/2018', 'Electra')
("Electra Townie Original 21D Ladies' - 2018", 'Electra')
("Electra Townie Original 21D EQ Ladies' - 2018", 'Electra')
("Electra Townie Commute 8D Ladies' - 2018", 'Electra')
('Electra Townie Commute 8D - 2018', 'Electra')
('Electra Townie Commute 27D Ladies - 2018', 'Electra')
("Electra Townie Balloon 8D EQ Ladies' - 2016/2017/2018", 'Electra')
('Electra Townie Original 21D - 2016', 'Electra')
('Electra Cruiser 1 (24-Inch) - 2016', 'Electra')
("Electra Girl's Hawaii 1 (16-inch) - 2015/2016", 'Electra')
('Electra Moto 1 - 2016', 'Electra')
('Electra Townie Original 7D EQ - 2016', 'Electra')
('Electra Townie Balloon 8D EQ - 2016/2017/2018', 'Electra')
('Electra Relic 3i - 2018', 'Electra')
("Electra

In [7]:
# Запрос №2 Список всех активных сотрудников и наименования магазинов, в которых они работают.
with engine.connect() as connection:
    query = text("""
        SELECT 
            CONCAT(s.first_name, ' ', s.last_name), 
            (SELECT ss.store_name FROM stores ss WHERE ss.store_id = s.store_id) AS store_name 
        FROM 
            staffs AS s 
        WHERE 
            s.active = 1
    """)  # Запрос SQL
    result = connection.execute(query)  # Выполняем запрос
    
    # Выводим результаты на экран
    for row in result:
        print(row)

('Fabiola Jackson', 'Santa Cruz Bikes')
('Mireya Copeland', 'Santa Cruz Bikes')
('Genna Serrano', 'Santa Cruz Bikes')
('Virgie Wiggins', 'Santa Cruz Bikes')
('Jannette David', 'Baldwin Bikes')
('Marcelene Boyer', 'Baldwin Bikes')
('Venita Daniel', 'Baldwin Bikes')
('Kali Vargas', 'Rowlett Bikes')
('Layla Terrell', 'Rowlett Bikes')
('Bernardine Houston', 'Rowlett Bikes')


In [8]:
#Запрос №3 Список всех покупателей выбранного магазина (Santa Cruz Bikes) с указанием их полных имен, электронной почты и номера телефона.
with engine.connect() as connection:
    query = text("""
        SELECT 
            CONCAT(c.first_name, ' ', c.last_name) full_name, 
            c.phone, 
            c.email, 
            s.store_name 
        FROM 
            customers c, 
            orders o, 
            stores s
        WHERE 
            s.store_id = o.store_id 
        AND 
            o.customer_id = c.customer_id 
        AND 
            s.store_name = 'Santa Cruz Bikes'
    """)  # Запрос SQL
    result = connection.execute(query)  # Выполняем запрос
    
    # Выводим результаты на экран
    for row in result:
        print(row)

('Johnathan Velazquez', None, 'johnathan.velazquez@hotmail.com', 'Santa Cruz Bikes')
('Nova Hess', None, 'nova.hess@msn.com', 'Santa Cruz Bikes')
('Neil Mccall', None, 'neil.mccall@gmail.com', 'Santa Cruz Bikes')
('Marvin Mullins', '(619) 635-2027', 'marvin.mullins@aol.com', 'Santa Cruz Bikes')
('Maribel William', None, 'maribel.william@aol.com', 'Santa Cruz Bikes')
('Lea Key', None, 'lea.key@yahoo.com', 'Santa Cruz Bikes')
('Sindy Anderson', None, 'sindy.anderson@gmail.com', 'Santa Cruz Bikes')
('Lanita Burton', None, 'lanita.burton@msn.com', 'Santa Cruz Bikes')
('Norine Huffman', None, 'norine.huffman@aol.com', 'Santa Cruz Bikes')
('Randee Pitts', None, 'randee.pitts@msn.com', 'Santa Cruz Bikes')
('Tangela Hurley', None, 'tangela.hurley@msn.com', 'Santa Cruz Bikes')
('Drucilla Gilliam', None, 'drucilla.gilliam@hotmail.com', 'Santa Cruz Bikes')
('Kimbery Nieves', None, 'kimbery.nieves@msn.com', 'Santa Cruz Bikes')
('Darren Witt', None, 'darren.witt@aol.com', 'Santa Cruz Bikes')
('Twan

In [9]:
# Запрос №4 Подсчет количества продуктов в каждой категории
with engine.connect() as connection:
    query = text("""
        SELECT 
            c.category_name, 
            (SELECT 
                COUNT(p.product_id) 
            FROM 
                products p 
            WHERE 
                p.category_id = c.category_id)  AS count_products  
        FROM categories c
    """)  # Запрос SQL
    result = connection.execute(query)  # Выполняем запрос
    
    # Выводим результаты на экран
    for row in result:
        print(row)


('Children Bicycles', 59)
('Comfort Bicycles', 30)
('Cruisers Bicycles', 78)
('Cyclocross Bicycles', 10)
('Electric Bikes', 24)
('Mountain Bikes', 60)
('Road Bikes', 60)


In [10]:
# Запрос №5 Список клиентов с указанием количества заказов.
with engine.connect() as connection:
    query = text("""
        SELECT 
            CONCAT(c.first_name, ' ', c.last_name) AS customer_name, 
            (SELECT 
                COUNT(o.order_id) 
            FROM 
                orders o 
            WHERE 
                o.customer_id = c.customer_id) AS order_count 
        FROM 
            customers c 
        ORDER BY order_count DESC
    """)  # Запрос SQL
    result = connection.execute(query)  # Выполняем запрос
    
    # Выводим результаты на экран
    for row in result:
        print(row)

('Debra Burks', 3)
('Kasha Todd', 3)
('Tameka Fisher', 3)
('Daryl Spence', 3)
('Charolette Rice', 3)
('Lyndsey Bean', 3)
('Latasha Hays', 3)
('Jacquline Duncan', 3)
('Genoveva Baldwin', 3)
('Pamelia Newman', 3)
('Deshawn Mendoza', 3)
('Robby Sykes', 3)
('Lashawn Ortiz', 3)
('Garry Espinoza', 3)
('Linnie Branch', 3)
('Emmitt Sanchez', 3)
('Caren Stephens', 3)
('Georgetta Hardin', 3)
('Lizzette Stein', 3)
('Aleta Shepard', 3)
('Tobie Little', 3)
('Corene Wall', 3)
('Jamaal Albert', 3)
('Williemae Holloway', 3)
('Araceli Golden', 3)
('Deloris Burke', 3)
('Ronna Butler', 3)
('Mozelle Carter', 3)
('Monika Berg', 3)
('Bridgette Guerra', 3)
('Cleotilde Booth', 3)
('Saturnina Garner', 3)
('Lolita Mosley', 3)
('Elinore Aguilar', 3)
('Bobbie Foster', 3)
('Lorrie Becker', 3)
('Jayne Kirkland', 3)
('Keri Bridges', 3)
('Merlene Vinson', 3)
('Adelle Larsen', 2)
('Kaylee English', 2)
('Regenia Vaughan', 2)
('Theo Reese', 2)
('Santos Valencia', 2)
('Jeanice Frost', 2)
('Syreeta Hendricks', 2)
('Brittn

In [11]:
# Запрос №6 Полное имя и общея количество заказов клиентов, которые хотя бы 1 раз сделали заказ.
with engine.connect() as connection:
    query = text("""
        SELECT 
            (SELECT 
                CONCAT(c.first_name, ' ', c.last_name) 
            FROM 
                customers c 
            WHERE 
                c.customer_id = o.Customer_id) AS cus_full_name, 
            COUNT(o.customer_id) AS count_orders 
        FROM 
            orders o 
        GROUP BY 
            o.customer_id
        ORDER BY 
            cus_full_name 
    """)  # Запрос SQL
    result = connection.execute(query)  # Выполняем запрос
    
    # Выводим результаты на экран
    for row in result:
        print(row)

('Aaron Knapp', 1)
('Abbey Pugh', 1)
('Abby Gamble', 2)
('Abram Copeland', 1)
('Adam Henderson', 1)
('Adam Thornton', 1)
('Addie Hahn', 1)
('Adelaida Hancock', 1)
('Adelle Larsen', 2)
('Adena Blake', 1)
('Adrien Hunter', 1)
('Adriene Rivera', 1)
('Adriene Rollins', 1)
('Afton Juarez', 1)
('Agatha Daniels', 1)
('Agatha Melton', 1)
('Agnes Sims', 1)
('Agustina Lawrence', 1)
('Ai Forbes', 1)
('Aida Koch', 1)
('Aide Franco', 1)
('Aileen Marquez', 1)
('Aimee Merritt', 1)
('Aisha Woods', 1)
('Alane Kennedy', 1)
('Alane Mccarty', 1)
('Alane Munoz', 1)
('Alanna Barry', 1)
('Alden Atkinson', 1)
('Alec Peck', 1)
('Alejandrina Hodges', 1)
('Alejandro Haney', 1)
('Alejandro Norman', 1)
('Alesia Horne', 1)
('Aleta Mack', 1)
('Aleta Shepard', 3)
('Aleta Stone', 1)
('Alexandria Zamora', 1)
('Alexis Mack', 1)
('Alfredo Dodson', 1)
('Alica Hunter', 2)
('Alina Mcleod', 1)
('Alishia Elliott', 1)
('Alisia Albert', 1)
('Alissa Craft', 1)
('Alissa Hood', 1)
('Alita Salinas', 1)
('Allie Conley', 1)
('Alline 

----------------------------------------------------

Задача 2 Работа с pySpark

In [12]:
# Импорт необходимых библиотек 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

In [13]:
# Запрос №1 Расчета общего объема продаж по каждому продукту (с учетом количества продукта, его цены по прейскуранту и скидки).
# Создание сеанса Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Query#1") \
    .getOrCreate()

# Загрузка данных из файлов CSV
order_items = spark.read.csv("data/order_items.csv", header=True, inferSchema=True)
products = spark.read.csv("data/products.csv", header=True, inferSchema=True)

# Регистрация DataFrame в качестве временной таблицы для использования в SQL
order_items.createOrReplaceTempView("order_items")
products.createOrReplaceTempView("products")

# Выполнение SQL-запроса
result = spark.sql("""
    SELECT 
        p.product_name,
        ROUND(SUM(oi.quantity * (oi.list_price * (1 - oi.discount / 100))),2) AS total_sales
    FROM 
        order_items oi
    JOIN 
        products p ON oi.product_id = p.product_id
    GROUP BY 
        p.product_name
    ORDER BY 
        total_sales DESC
""")

# Вывод результата
result.show(truncate=False)

# Закрытие сеанса Spark
spark.stop()

+-------------------------------------+-----------+
|product_name                         |total_sales|
+-------------------------------------+-----------+
|Trek Slash 8 27.5 - 2016             |615394.06  |
|Trek Conduit+ - 2016                 |434541.05  |
|Trek Fuel EX 8 29 - 2016             |414236.31  |
|Surly Straggler 650b - 2016          |253558.85  |
|Trek Domane SLR 6 Disc - 2017        |236250.42  |
|Surly Straggler - 2016               |227461.05  |
|Trek Remedy 29 Carbon Frameset - 2016|224782.57  |
|Trek Powerfly 8 FS Plus - 2017       |204832.09  |
|Trek Madone 9.2 - 2017               |194808.61  |
|Trek Silque SLR 8 Women's - 2017     |188359.96  |
|Trek Silque SLR 7 Women's - 2017     |179830.5   |
|Trek Fuel EX 9.8 27.5 Plus - 2017    |174741.2   |
|Electra Townie Original 7D EQ - 2016 |173808.82  |
|Heller Shagamaw Frame - 2016         |170215.24  |
|Trek Fuel EX 9.8 29 - 2017           |164824.17  |
|Electra Townie Original 21D - 2016   |158791.57  |
|Trek Boone 

In [14]:
# Запрос №2 Расчет количества заказов по каждому статусу заказа.
# Создание сеанса Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Query#2") \
    .getOrCreate()

# Загрузка данных из файла CSV
orders = spark.read.csv("data/orders.csv", header=True, inferSchema=True)


# Регистрация DataFrame в качестве временной таблицы для использования в SQL
orders.createOrReplaceTempView("orders")


# Выполнение SQL-запроса
result = spark.sql("""
    SELECT 
        o.order_status, 
        count(o.order_id) AS count_orders 
    FROM 
        orders o
    GROUP BY 
        o.order_status
    ORDER BY 
        o.order_status
""")

# Вывод результата
result.show(truncate=False)

# Закрытие сеанса Spark
spark.stop()

+------------+------------+
|order_status|count_orders|
+------------+------------+
|1           |62          |
|2           |63          |
|3           |45          |
|4           |1445        |
+------------+------------+



In [15]:
# Запрос №3 Расчет общей суммы продаж за каждый месяц. 
# Так как не вижу в файле даты продажи (есть требуемая дата и дата отгрузки), 
# а политика магазина "утром деньги-вечером велик", считаем датой продажи - дату заказа (order_date),
# при этом статус заказа берем 4 ("завершено" если верить источнику)

# Создание сеанса Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Query#3") \
    .getOrCreate()

# Загрузка данных из файлов CSV
order_items = spark.read.csv("data/order_items.csv", header=True, inferSchema=True)
orders = spark.read.csv("data/orders.csv", header=True, inferSchema=True)

# Регистрация DataFrame в качестве временной таблицы для использования в SQL
order_items.createOrReplaceTempView("order_items")
orders.createOrReplaceTempView("orders")


# Выполнение SQL-запроса
result = spark.sql("""
    SELECT
        DATE_FORMAT(o.order_date, 'yyyy-MM') as month_order, 
        ROUND(SUM(oi.quantity * (oi.list_price * (1 - oi.discount / 100))),2) as total_sales   
    FROM 
        order_items oi
    JOIN  
        orders o on oi.order_id = o.order_id
    WHERE 
        o.order_status = 4
    GROUP BY 
        month_order
    ORDER BY 
        month_order DESC
""")

# Вывод результата
result.show(truncate=False)

# Закрытие сеанса Spark
spark.stop()

+-----------+-----------+
|month_order|total_sales|
+-----------+-----------+
|2018-03    |406274.1   |
|2018-02    |220013.79  |
|2018-01    |425853.0   |
|2017-12    |290707.7   |
|2017-11    |301706.42  |
|2017-10    |337531.67  |
|2017-09    |329028.85  |
|2017-08    |303501.8   |
|2017-07    |238083.64  |
|2017-06    |415640.48  |
|2017-05    |291715.08  |
|2017-04    |246478.3   |
|2017-03    |346965.15  |
|2017-02    |313232.78  |
|2017-01    |316641.39  |
|2016-12    |212241.4   |
|2016-11    |198419.58  |
|2016-10    |234822.05  |
|2016-09    |301061.58  |
|2016-08    |238640.24  |
+-----------+-----------+
only showing top 20 rows



In [17]:
# Запрос №4 Расчет ТОП5 клиентов которые потратили больше всего денег. 
# Опять берем только завершенные заказы, статус 4 

# Создание сеанса Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Query#4") \
    .getOrCreate()

# Загрузка данных из файлов CSV
order_items = spark.read.csv("data/order_items.csv", header=True, inferSchema=True)
orders = spark.read.csv("data/orders.csv", header=True, inferSchema=True)
customers = spark.read.csv("data/customers.csv", header=True, inferSchema=True)


# Регистрация DataFrame в качестве временных таблиц для использования в SQL
order_items.createOrReplaceTempView("order_items")
orders.createOrReplaceTempView("orders")
customers.createOrReplaceTempView("customers")


# Выполнение SQL-запроса
result = spark.sql("""
    SELECT  
	    CONCAT(c.first_name, ' ', c.last_name) as full_name,
        ROUND(SUM(oi.quantity * (oi.list_price * (1 - oi.discount / 100))),2) total_sales   
    FROM 
        customers c, 
        orders o, 
        order_items oi
    WHERE 
        oi.order_id = o.order_id 
    AND 
        o.customer_id = c.customer_id
    AND 
        o.order_status = 4
    GROUP BY 
        full_name
    ORDER BY 
        total_sales desc
    LIMIT 5
""")

# Вывод результата
result.show(truncate=False)


+--------------+-----------+
|full_name     |total_sales|
+--------------+-----------+
|Melanie Hayes |29129.91   |
|Shena Carter  |27591.67   |
|Abram Copeland|26890.85   |
|Adena Blake   |22963.24   |
|Brigid Sharp  |22283.44   |
+--------------+-----------+

